In [1]:
! source ../env_btp/bin/activate

## Preprocessing

In [2]:
from preprocessing.pan21 import read_dataset

path = './data/pan21/train/en'

In [3]:
data = read_dataset(path)

In [4]:
data.keys()

dict_keys([0, 1])

In [5]:
len(data[0]), len(data[0][0])

(100, 200)

## Stats

In [6]:
from eda.get_stats import *

ModuleNotFoundError: No module named 'spacy'

In [ ]:
hashtags = count_hashtags(data)
urls = count_urls(data)
users = count_users(data)
rt = count_rt(data)


# print out
print('#HASHTAG#:', hashtags)
print('#URL#:', urls)
print('#USER#:', users)
print('##RT##:', rt)


#HASHTAG#: {0: 3757, 1: 3392}
#URL#: {0: 8571, 1: 6768}
#USER#: {0: 9723, 1: 11571}
##RT##: {0: 7633, 1: 6090}


In [ ]:
uppercase_chars = count_uppercase_chars(data)
min_chars = count_min_chars(data)
avg_min_chars = count_avg_min_chars(data)
max_chars = count_max_chars(data)
avg_max_chars = count_avg_max_chars(data)
chars = count_chars(data)

# print out
print('number of upper-case characters:', uppercase_chars)
print('min min characters:', min_chars)
print('avg min characters:', avg_min_chars)
print('max max characters:', max_chars)
print('avg max characters:', avg_max_chars)
print('number of characters:', chars)





number of upper-case characters: {0: 71025, 1: 75867}
min min characters: {0: 4, 1: 5}
avg min characters: {0: 10.28, 1: 10.55}
max max characters: {0: 143, 1: 148}
avg max characters: {0: 125.08, 1: 128.2}
number of characters: {0: 1109779, 1: 1134313}


In [ ]:
uppercase_words = count_uppercase_words(data)
min_words = count_min_words(data)
avg_min_words = count_avg_min_words(data)
max_words = count_max_words(data)
avg_max_words = count_avg_max_words(data)
words = count_words(data)

# print out
print('number of upper-case words:', uppercase_words)
print('min min words:', min_words)
print('avg min words:', avg_min_words)
print('max max words:', max_words)
print('avg max words:', avg_max_words)
print('number of words:', words)


number of upper-case words: {0: 43584, 1: 44012}
min min words: {0: 1, 1: 1}
avg min words: {0: 2.53, 1: 2.68}
max max words: {0: 31, 1: 32}
avg max words: {0: 24.85, 1: 25.58}
number of words: {0: 207317, 1: 213744}


In [ ]:
stop_words = count_stopwords(data)

# print out
print('number of stop-words:', stop_words)

In [ ]:
uppercase_words = count_uppercase_words(data)
